In [1]:
# import TensorFlow as TF
import pandas as pd
import math
import matplotlib.pylab as plt
import numpy as np

from datetime import datetime

from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer # used for handling missing data
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_absolute_error


In [3]:

df = pd.read_csv('data.csv')
data_top = df.head()
copy = df.copy() 


def create_dict(target_var, df): 
    org_ids = list(df[target_var]) 
    list_set = set(org_ids) # get unique elements
    unique = list(list_set) # place them in a list 

    df = df.groupby([target_var]) 

    dictionary = {} 

    for item in unique:
        df_id = df.get_group(item)
        dictionary[item] = df_id
    
    return dictionary

def df_clean(df, column_name):
    df = df.reset_index(drop=True)
    
    for i in range(1, len(df[column_name])-2):
        if math.isnan(df.at[i, column_name]):
            if not math.isnan(df.at[i-1, column_name]):
                if not math.isnan(df.at[i+1, column_name]):
                    mean = (df.at[i+1, column_name] + df.at[i-1, column_name])/2
                    df.at[i, column_name] = mean
    
    df = df[df[column_name].notna()]
    return df

def dict_clean_final(dictionary, column_name):
    dict_clean = {}
    for key in dictionary:
        dict_clean[key] = df_clean(dictionary[key], column_name)
    return dict_clean

In [4]:
dict_id = create_dict('organization_id', copy)
dict_id = dict_clean_final(dict_id, 'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1')

In [12]:
#Adds column to df which contains the scaled Emissions based on sector (Assuming sector input) 
def scale_emissions(df):
    df.loc[:,'CO2toRevenue'] = df.loc[:,'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1']*(df.loc[:,'EmissionIndicators.TotalCO2EquivalentsEmissionsToRevenues']/df.loc[:,'EmissionDataPoints.CO2EquivalentsEmissionTotal'])
    return df

#Takes threshold in order to determine if you want to use sector or org_id
# returns true if you need to use sector

def sector_or_orgid(orgid, df, threshold = 10):
#     len(df[df['organization_id']==orgid])
    return len(df[df.loc[:,'organization_id']==orgid]) >= threshold

# func: makes predictions for a given org_id
# params: a dict of all org_id's as keys with their corresponding df's as values, and a specific org_id
# returns: the predictions and a plot of the predictions 
def predict(dictionary, orgid): 
    # decide wether to use sector or org_id for predictions:
    if sector_or_orgid(orgid, dictionary[orgid]):      
        print('predictions based on orgid')
        df = dictionary[orgid]
        df = df_clean(df, 'StatementDetails.FinancialPeriodFiscalYear')
        df = df_clean(df, 'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1')
        X = df.loc[:, ['StatementDetails.FinancialPeriodFiscalYear']]
        X = np.array(X)
        Y = df.loc[:,'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1']
        title = 'Plot of emissions for company: ' + str(orgid)
        y_label = "CO2 emission"
    else: 
        print('predictions based on sector (using scaled data)')
        sector = copy['GTAP sector'].loc[orgid]
        sector_dict = create_dict('GTAP sector', copy)
        # adding a column with scaled emissions
        df_sector = scale_emissions(sector_dict[sector])
        df_sector = df_clean(df_sector, 'CO2toRevenue')
        X = df_sector.loc[:, ['StatementDetails.FinancialPeriodFiscalYear']]
        # use scaled emissions as target instead of scope1
        Y = df_sector.loc[:,'CO2toRevenue']
        title = 'Emissions for all companies in the sector: ' + sector
        y_label = 'CO2 emission per revenue'
    
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.33, random_state=42)
    degree = 1
    polyreg=make_pipeline(PolynomialFeatures(degree), LinearRegression())
    polyreg.fit(X_train,Y_train)
    predictions = polyreg.predict(X_test)
    mean_abs_error = mean_absolute_error(Y_test, predictions)
#     print("Mean absolute error: %.2f" % mean_absolute_error(Y_test, predictions))
    # The coefficient of determination: 1 is perfect prediction
#     print("Coefficient of determination: %.2f" % r2_score(Y_test, predictions))
#     plt.plot(X_test, predictions, label='prediction')
#     plt.scatter(X_test, Y_test, label='test data')
#     #X_test[:, 0]
#     plt.xlabel("year")
#     plt.ylabel(y_label)
#     plt.title(title)
#     plt.legend()
#     plt.show()
    return predictions, mean_abs_error


In [13]:
abs_error_list = []
for key in dict_id:
    if len(dict_id[key]) > 0:
        abs_error_list.append(predict(dict_id, key)[1])
    
print(abs_error_list)

predictions based on orgid
predictions based on sector (using scaled data)
predictions based on sector (using scaled data)


/var/folders/s_/qtbdy3px3m3_6k7xydrkhbsh0000gn/T/ipykernel_7686/191058123.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'CO2toRevenue'] = df.loc[:,'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1']*(df.loc[:,'EmissionIndicators.TotalCO2EquivalentsEmissionsToRevenues']/df.loc[:,'EmissionDataPoints.CO2EquivalentsEmissionTotal'])
/var/folders/s_/qtbdy3px3m3_6k7xydrkhbsh0000gn/T/ipykernel_7686/191058123.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'CO2toRevenue'] = df.loc[:,

predictions based on sector (using scaled data)
predictions based on sector (using scaled data)


/var/folders/s_/qtbdy3px3m3_6k7xydrkhbsh0000gn/T/ipykernel_7686/191058123.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'CO2toRevenue'] = df.loc[:,'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1']*(df.loc[:,'EmissionIndicators.TotalCO2EquivalentsEmissionsToRevenues']/df.loc[:,'EmissionDataPoints.CO2EquivalentsEmissionTotal'])
/var/folders/s_/qtbdy3px3m3_6k7xydrkhbsh0000gn/T/ipykernel_7686/191058123.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'CO2toRevenue'] = df.loc[:,

predictions based on sector (using scaled data)
predictions based on sector (using scaled data)


...

predictions based on orgid
predictions based on sector (using scaled data)
predictions based on sector (using scaled data)
predictions based on orgid
predictions based on sector (using scaled data)
predictions based on sector (using scaled data)


/var/folders/s_/qtbdy3px3m3_6k7xydrkhbsh0000gn/T/ipykernel_7686/191058123.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'CO2toRevenue'] = df.loc[:,'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1']*(df.loc[:,'EmissionIndicators.TotalCO2EquivalentsEmissionsToRevenues']/df.loc[:,'EmissionDataPoints.CO2EquivalentsEmissionTotal'])
/var/folders/s_/qtbdy3px3m3_6k7xydrkhbsh0000gn/T/ipykernel_7686/191058123.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'CO2toRevenue'] = df.loc[:,

predictions based on sector (using scaled data)


/var/folders/s_/qtbdy3px3m3_6k7xydrkhbsh0000gn/T/ipykernel_7686/191058123.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'CO2toRevenue'] = df.loc[:,'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1']*(df.loc[:,'EmissionIndicators.TotalCO2EquivalentsEmissionsToRevenues']/df.loc[:,'EmissionDataPoints.CO2EquivalentsEmissionTotal'])


ValueError: With n_samples=0, test_size=0.33 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [15]:
print(abs_error_list.mean())

AttributeError: 'list' object has no attribute 'mean'